In [1]:
# Environment setup
import os
import json
import json5
import nest_asyncio
from dotenv import load_dotenv

nest_asyncio.apply()
load_dotenv(".env")  # Load your OpenAI API key from .env file

from pathlib import Path
import pathlib

from lmi import CommonLLMNames
from paperqa.settings import Settings
from aviary.envs.litqa.env import GradablePaperQAEnvironment
from ldp.agent import SimpleAgent
from ldp.alg import Evaluator, EvaluatorConfig, MeanMetricsCallback
from aviary.env import TaskDataset
from aviary.envs.litqa.task import TASK_DATASET_NAME

/Users/jiali03/Desktop/xc392/paper-qa/.venv/lib/python3.13/site-packages/pybtex/plugin/__init__.py:26: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


## Set the model, settings and dataset

In [9]:
# # Set up model and local paper directory
# llm_openai = CommonLLMNames.OPENAI_TEST.value
# paper_dir = Path("./litqa_pdfs")
# settings = Settings(paper_directory=paper_dir)

# # Load question IDs from json5 split
# with open("2024-10-16_litqa2-splits.json5", "r") as f:
#     split_dict = json5.load(f)
# eval_ids = set(split_dict["eval"]["question_ids"])

# # Load litqa-v2-public.jsonl and filter by eval split
# questions = []
# with open("litqa-v2-public.jsonl", "r") as f:
#     for line in f:
#         q = json5.loads(line)
#         if q["id"] in eval_ids:
#             questions.append(q)

# # Manually construct list of environments
# envs = [
#     GradablePaperQAEnvironment(question=q, settings=settings)
#     for q in questions
# ]

# dataset = envs

In [2]:
# Set the model (OpenAI GPT-4)
llm_openai = CommonLLMNames.OPENAI_TEST.value

In [3]:
from paperqa.prompts import (
    CONTEXT_INNER_PROMPT,
    CONTEXT_OUTER_PROMPT,
    citation_prompt,
    default_system_prompt,
    env_reset_prompt,
    env_system_prompt,
    qa_prompt,
    select_paper_prompt,
    structured_citation_prompt,
    summary_json_prompt,
    summary_json_system_prompt,
    summary_prompt,
)
from paperqa.settings import (
    AgentSettings,
    AnswerSettings,
    IndexSettings,
    ParsingSettings,
    PromptSettings,
    Settings,
)

settings = Settings(
    llm=llm_openai,
    llm_config={
        "model_list": [
            {
                "model_name": llm_openai,
                "litellm_params": {
                    "model": llm_openai,
                    "temperature": 0.1,
                    "max_tokens": 4096,
                },
            }
        ],
        "rate_limit": {
            llm_openai: "30000 per 1 minute",
        },
    },
    summary_llm=llm_openai,
    summary_llm_config={
        "rate_limit": {
            llm_openai: "30000 per 1 minute",
        },
    },
    embedding="text-embedding-3-small",
    embedding_config={},
    temperature=0.1,
    batch_size=1,
    verbosity=1,
    manifest_file=pathlib.Path.cwd().joinpath("litqa_pdfs/manifest.yaml"),
    paper_directory=pathlib.Path.cwd().joinpath("litqa_pdfs"),
    index_directory=pathlib.Path.cwd().joinpath("litqa_pdfs/index"),
    answer=AnswerSettings(
        evidence_k=10,
        evidence_detailed_citations=True,
        evidence_retrieval=True,
        evidence_summary_length="about 100 words",
        evidence_skip_summary=False,
        answer_max_sources=5,
        max_answer_attempts=None,
        answer_length="about 200 words, but can be longer",
        max_concurrent_requests=10,
    ),
    parsing=ParsingSettings(
        chunk_size=5000,
        overlap=250,
        citation_prompt=citation_prompt,
        structured_citation_prompt=structured_citation_prompt,
    ),
    prompts=PromptSettings(
        summary=summary_prompt,
        qa=qa_prompt,
        select=select_paper_prompt,
        pre=None,
        post=None,
        system=default_system_prompt,
        use_json=True,
        summary_json=summary_json_prompt,
        summary_json_system=summary_json_system_prompt,
        context_outer=CONTEXT_OUTER_PROMPT,
        context_inner=CONTEXT_INNER_PROMPT,
    ),
    agent=AgentSettings(
        agent_llm=llm_openai,
        agent_llm_config={
            "model_list": [
                {
                    "model_name": llm_openai,
                    "litellm_params": {
                        "model": llm_openai,
                    },
                }
            ],
            "rate_limit": {
                llm_openai: "30000 per 1 minute",
            },
        },
        agent_prompt=env_reset_prompt,
        agent_system_prompt=env_system_prompt,
        search_count=8,
        index=IndexSettings(
            paper_directory=pathlib.Path.cwd().joinpath("litqa_pdfs"),
            index_directory=pathlib.Path.cwd().joinpath("litqa_pdfs/index"),
        ),
    ),
)

In [4]:
# Load the LitQA v2 dataset (eval split)
dataset = TaskDataset.from_name(TASK_DATASET_NAME, settings=settings)

/Users/jiali03/Desktop/xc392/paper-qa/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import json
from uuid import UUID

records = []
for i in range(len(dataset)):
    env = dataset.get_new_env_by_idx(i)
    q = env._query

    record = {
        "id": str(q.question_id) if isinstance(q.question_id, UUID) else q.question_id,
        "question": q.question,
        "options": q.options,
        "correct_answer": q.ideal_answer,
        "sources": env.sources,
    }
    records.append(record)

with open("data/litqa_eval_repro_v1.json", "w", encoding="utf-8") as f:
    json.dump(records, f, indent=2, ensure_ascii=False)

print(f"Saved {len(records)} questions to litqa_eval_repro_v1.json")

Saved 40 questions to litqa_eval_repro_v1.json


## Check dataset

In [ ]:
# import fitz  # PyMuPDF
# from pathlib import Path

# pdf_dir = Path("litqa_pdfs")
# bad_files = []

# for pdf_path in pdf_dir.glob("*.pdf"):
#     try:
#         doc = fitz.open(pdf_path)
#         text = ""
#         for page in doc:
#             text += page.get_text()
#         if len(text.strip()) < 100:
#             bad_files.append((pdf_path.name, "empty_text"))
#         doc.close()
#     except Exception as e:
#         bad_files.append((pdf_path.name, f"error: {e}"))

# print(f"Checked {len(list(pdf_dir.glob('*.pdf')))} files.")
# print(f"{len(bad_files)} files are likely unreadable:")
# for f, reason in bad_files:
#     print(f"{f}\t{reason}")


Checked 14 files.
0 files are likely unreadable:


In [ ]:
# from paperqa import Docs
# from pathlib import Path

# docs = Docs()
# pdf_dir = Path("litqa_pdfs")

# unreadable = []
# for f in pdf_dir.glob("*.pdf"):
#     try:
#         await docs.aadd(f)
#     except Exception as e:
#         unreadable.append((f.name, str(e)))

# print(f"Checked {len(list(pdf_dir.glob('*.pdf')))} files.")
# print(f"{len(unreadable)} files are likely unreadable:")
# for name, err in unreadable:
#     print(name, "|", err)


In [10]:
import json
import json5

with open("litqa_eval_repro.json", "r", encoding="utf-8") as f:
    repro_data = json.load(f)
repro_ids = set(item["id"] for item in repro_data)

with open("2024-10-16_litqa2-splits.json5", "r", encoding="utf-8") as f:
    split_data = json5.load(f)
eval_ids = set(split_data.get("eval", {}).get("question_ids", []))

only_in_repro = repro_ids - eval_ids
only_in_split = eval_ids - repro_ids
all_match = (only_in_repro == set()) and (only_in_split == set())

print(f"Is the same: {all_match}")
print(f"Only in repro.json: {len(only_in_repro)}")
print(f"Only in json5: {len(only_in_split)}")

if not all_match:
    print(f"Only in repro.json: {list(only_in_repro)[:3]}")
    print(f"Only in json5: {list(only_in_split)[:3]}")

Is the same: False
Only in repro.json: 31
Only in json5: 31
Only in repro.json: ['4bb69c9d-2485-42d2-b8aa-aa647b407ca4', 'cdc80639-dc21-4337-bccc-d61d9aec6560', 'bca1be77-208b-4d57-ac29-05aa6d58bdbf']
Only in json5: ['c75879f4-9329-4594-8f6a-c4d421f2439f', '5806ed2a-1005-44f1-8d70-332048fea8f4', '8d12cb90-73c6-4cc1-9f83-0fa9fd822e92']


Every time the eval set is random? Yes!

In [19]:
# Load the LitQA v2 dataset (eval split)
dataset2 = TaskDataset.from_name(TASK_DATASET_NAME, settings=settings)

In [20]:
import json
from uuid import UUID

records = []
for i in range(len(dataset2)):
    env = dataset2.get_new_env_by_idx(i)
    q = env._query

    record = {
        "id": str(q.question_id) if isinstance(q.question_id, UUID) else q.question_id,
        "question": q.question,
        "options": q.options,
        "correct_answer": q.ideal_answer,
        "sources": env.sources,
    }
    records.append(record)

with open("litqa_eval_repro2.json", "w", encoding="utf-8") as f:
    json.dump(records, f, indent=2, ensure_ascii=False)

print(f"Saved {len(records)} questions to litqa_eval_repro2.json")

Saved 40 questions to litqa_eval_repro2.json


In [ ]:
import json
import json5

# with open("litqa_eval_repro.json", "r", encoding="utf-8") as f:
#     repro_data = json.load(f)
with open("litqa_eval_repro2.json", "r", encoding="utf-8") as f:
    repro_data = json.load(f)
repro_ids = set(item["id"] for item in repro_data)

with open("2024-10-16_litqa2-splits.json5", "r", encoding="utf-8") as f:
    split_data = json5.load(f)

train_ids = set(split_data.get("train", {}).get("question_ids", []))
eval_ids = set(split_data.get("eval", {}).get("question_ids", []))
combined_ids = train_ids | eval_ids

is_subset = repro_ids.issubset(combined_ids)

print(f"Is the subset of train.question_ids + eval.question_ids: {is_subset}")
print(f"Number of repro IDs: {len(repro_ids)}")
print(f"Number of train and eval IDs: {len(combined_ids)}")
print(f"Not in trai or eval ID: {len(repro_ids - combined_ids)}")

if not is_subset:
    print(list(repro_ids - combined_ids)[:5])


Is the subset of train.question_ids + eval.question_ids: True
Number of repro IDs: 40
Number of train and eval IDs: 199
Not in trai or eval ID: 0


In [22]:
import json

# Load IDs from litqa_eval_repro2.json
with open("litqa_eval_repro2.json", "r", encoding="utf-8") as f:
    eval_data = json.load(f)
eval_ids = set(item["id"] for item in eval_data)

# Load IDs from litqa-v2-public.jsonl
jsonl_ids = set()
with open("litqa-v2-public.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        entry = json.loads(line)
        jsonl_ids.add(entry["id"])

# Check if eval_ids is a subset of jsonl_ids
is_subset = eval_ids.issubset(jsonl_ids)

print(f"Is litqa_eval_repro2.json a subset of litqa-v2-public.jsonl: {is_subset}")
print(f"Number of eval IDs: {len(eval_ids)}")
print(f"Number of JSONL IDs: {len(jsonl_ids)}")
print(f"Number of IDs not found in JSONL: {len(eval_ids - jsonl_ids)}")

# Show a few missing IDs if not a subset
if not is_subset:
    print("Missing ID examples:")
    print(list(eval_ids - jsonl_ids)[:5])


Is litqa_eval_repro2.json a subset of litqa-v2-public.jsonl: True
Number of eval IDs: 40
Number of JSONL IDs: 199
Number of IDs not found in JSONL: 0


## Generate index

In [ ]:
from paperqa import ask

response = ask("The NCAN-ELS domain has been shown to interact with what percent of inhibitory synapses?", settings=settings)

PaperQA version: 5.21.1.dev17+g2cf0d6c


[00:03:18] Beginning agent 'ToolSelector' run with question 'The NCAN-ELS domain has been shown to interact with   
           what percent of inhibitory synapses?' and full settings {'llm': 'gpt-4o-mini-2024-07-18', 'llm_config': 
           {'model_list': [{'model_name': 'gpt-4o-mini-2024-07-18', 'litellm_params': {'model':                    
           'gpt-4o-mini-2024-07-18', 'temperature': 0.1, 'max_tokens': 4096}}], 'rate_limit':                      
           {'gpt-4o-mini-2024-07-18': '30000 per 1 minute'}}, 'summary_llm': 'gpt-4o-mini-2024-07-18',             
           'summary_llm_config': {'rate_limit': {'gpt-4o-mini-2024-07-18': '30000 per 1 minute'}}, 'embedding':    
           'text-embedding-3-small', 'embedding_config': {}, 'temperature': 0.1, 'batch_size': 1,                  
           'texts_index_mmr_lambda': 1.0, 'verbosity': 1, 'answer': {'evidence_k': 10,                             
           'evidence_detailed_citations': True, 'evidence_retrieval': True, 'evidence_summary_length': 'about 100  
           words', 'evidence_skip_summary': False, 'answer_max_sources': 5, 'max_answer_attempts': None,           
           'answer_length': 'about 200 words, but can be longer', 'max_concurrent_requests': 10,                   
           'answer_filter_extra_background': False, 'get_evidence_if_no_contexts': True}, 'parsing': {'chunk_size':
           5000, 'page_size_limit': 1280000, 'pdfs_use_block_parsing': False, 'use_doc_details': True, 'overlap':  
           250, 'citation_prompt': 'Provide the citation for the following text in MLA Format. Do not write an     
           introductory sentence. If reporting date accessed, the current year is 2025\n\n{text}\n\nCitation:',    
           'structured_citation_prompt': "Extract the title, authors, and doi as a JSON from this MLA citation. If 
           any field can not be found, return it as null. Use title, authors, and doi as keys, author's value      
           should be a list of authors. {citation}\n\nCitation JSON:", 'disable_doc_valid_check': False,           
           'defer_embedding': False, 'chunking_algorithm': <ChunkingOptions.SIMPLE_OVERLAP: 'simple_overlap'>,     
           'doc_filters': None, 'use_human_readable_clinical_trials': False}, 'prompts': {'summary': 'Summarize the
           excerpt below to help answer a question.\n\nExcerpt from                                                
           {citation}\n\n----\n\n{text}\n\n----\n\nQuestion: {question}\n\nDo not directly answer the question,    
           instead summarize to give evidence to help answer the question. Stay detailed; report specific numbers, 
           equations, or direct quotes (marked with quotation marks). Reply "Not applicable" if the excerpt is     
           irrelevant. At the end of your response, provide an integer score from 1-10 on a newline indicating     
           relevance to question. Do not explain your score.\n\nRelevant Information Summary ({summary_length}):', 
           'qa': 'Answer the question below with the context.\n\nContext (with relevance                           
           scores):\n\n{context}\n\n----\n\nQuestion: {question}\n\nWrite an answer based on the context. If the   
           context provides insufficient information reply "I cannot answer." For each part of your answer,        
           indicate which sources most support it via citation keys at the end of sentences, like                  
           {example_citation}. Only cite from the context above and only use the citation keys from the context. ##
           Valid citation examples: \n- Example2024Example pages 3-4 \n- Example2024 pages 3-4 \n- Example2024     
           pages 3-4, Example2024 pages 5-6 \n## Invalid citation examples: \n- Example2024Example pages 3-4 and   
           pages 4-5 \n- Example2024Example (pages 3-4) \n- Example2024Example pages 3-4, pages 5-6 \n-            
           Example2024Example et al. (2024) \n- Example\

           Invalid manifest file type: .yaml

           New file to index: 10.1186%2Fs12915-024-01908-y.pdf...

[00:03:19] New file to index: 10.1038%2Fs41467-024-47207-6.pdf...

...

Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30


[00:14:13] Status: Paper Count=8 | Relevant Papers=1 | Current Evidence=6 | Current Cost=$0.0032

           Completing 'The NCAN-ELS domain has been shown to interact with what percent of inhibitory synapses?' as
           'certain'.

           Finished agent 'ToolSelector' run with question 'The NCAN-ELS domain has been shown to interact with    
           what percent of inhibitory synapses?' and status success.

           Answer: The NCAN-ELS domain has been shown to interact with a significant percentage of inhibitory      
           synapses. Specifically, it colocalizes with 75% of inhibitory synapses and 50% of somatostatin-positive 
           (SST+) inhibitory synapses (irala2024astrocytesecretedneurocancontrols pages 10-11,                     
           irala2024astrocytesecretedneurocancontrols pages 12-13). Additionally, the study indicates that the NCAN
           C-terminal ELS domain plays a crucial role in the formation and function of these synapses, as evidenced
           by a marked decrease of about 30% in the density of inhibitory synapses when the NCAN-ELS domain was    
           deleted (irala2024astrocytesecretedneurocancontrols pages 8-8).                                         
                                                                                                                   
           The interaction of NCAN at inhibitory synapses underscores the importance of astrocyte-secreted         
           molecules in regulating synaptic balance, although the exact percentage of synapses specifically        
           associated with the NCAN-ELS domain is not directly stated in all excerpts                              
           (irala2024astrocytesecretedneurocancontrols pages 13-14). Overall, the findings highlight the           
           significant role of the NCAN C-terminal ELS domain in inhibitory synapse development and function.

## Run evaluation

In [ ]:
# # Evaluation
# metrics_callback = MeanMetricsCallback(eval_dataset=dataset)
# evaluator = Evaluator(
#     config=EvaluatorConfig(batch_size=3),
#     agent=SimpleAgent(),
#     dataset=dataset,
#     callbacks=[metrics_callback],
# )

# await evaluator.evaluate()
# print("Evaluation complete.")
# print("Mean evaluation metrics:")
# print(metrics_callback.eval_means)

Evaluation Iterations:   0% 0/14 [00:00<?, ?it/s]/Users/jianingli/Desktop/xc392/paper-qa/paperqa/llms.py:52: RuntimeWarning: divide by zero encountered in matmul
  return a @ b.T / norm_product
/Users/jianingli/Desktop/xc392/paper-qa/paperqa/llms.py:52: RuntimeWarning: overflow encountered in matmul
  return a @ b.T / norm_product
/Users/jianingli/Desktop/xc392/paper-qa/paperqa/llms.py:52: RuntimeWarning: invalid value encountered in matmul
  return a @ b.T / norm_product
00:16:14 - LiteLLM:WARNING: logging_callback_manager.py:124 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
00:16:14 - LiteLLM:WARNING: logging_callback_manager.py:124 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
00:16:14 - LiteLLM:WARNING: logging_callback_manager.py:124 - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
00:16:14 - LiteLLM:WARNING: logging_callback_manager.py:124 - Cannot add callback - w

Evaluation complete.
Mean evaluation metrics:
{'reward': 0.030000000000000006, 'truncation_rate': 0.0, 'avg_value': 0.0, 'num_steps': 8.675, 'failures': 0.0, 'total_paper_count': 10.5, 'relevant_paper_count': 0.175, 'evidence_count': 0.95, 'correct': 0.075, 'correct_unsure': 0.875}


In [ ]:
async def evaluate():
    metrics_callback = MeanMetricsCallback(eval_dataset=dataset)
    evaluator = Evaluator(
        config=EvaluatorConfig(batch_size=3),
        agent=SimpleAgent(),
        dataset=dataset,
        callbacks=[metrics_callback],
    )
    await evaluator.evaluate()
    print("Evaluation complete.")
    print("Mean evaluation metrics:")
    print(metrics_callback.eval_means)
    
await evaluate()

Run evaluation and save the result:

In [ ]:
async def evaluate():
    metrics_callback = MeanMetricsCallback(eval_dataset=dataset)
    evaluator = Evaluator(
        config=EvaluatorConfig(batch_size=3),
        agent=SimpleAgent(),
        dataset=dataset,
        callbacks=[metrics_callback],
    )
    await evaluator.evaluate()

    # Save result to JSON file
    with open("results/gpt_eval_results_v1.json", "w", encoding="utf-8") as f:
        json.dump(metrics_callback.eval_means, f, indent=2, ensure_ascii=False)

await evaluate()

Evaluation Iterations:   0% 0/14 [00:00<?, ?it/s]Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
Cannot add callback - would

[00:16:52] Starting paper search for 'Ingraham 2022 protein design symmetric space group'.

           paper_search for query 'Ingraham 2022 protein design symmetric space group' and offset 0 returned 8     
           papers.

           Status: Paper Count=8 | Relevant Papers=0 | Current Evidence=0 | Current Cost=$0.0000

Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30


...

[00:53:20] Generating answer for 'Q: Which reactive astrocyte marker has been shown to increase in expression in   
           RhoAnesKO mice?                                                                                         
                                                                                                                   
           Options:                                                                                                
           A) Nestin                                                                                               
           B) Synemin                                                                                              
           C) Vimentin                                                                                             
           D) GFAP                                                                                                 
           E) Insufficient information to answer this question'.

Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30


[00:53:24] Status: Paper Count=8 | Relevant Papers=1 | Current Evidence=8 | Current Cost=$0.0034

Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30


[00:53:25] Completing 'Q: Which reactive astrocyte marker has been shown to increase in expression in RhoAnesKO    
           mice?                                                                                                   
                                                                                                                   
           Options:                                                                                                
           A) Nestin                                                                                               
           B) Synemin                                                                                              
           C) Vimentin                                                                                             
           D) GFAP                                                                                                 
           E) Insufficient information to answer this question' as 'certain'.